# Домашнее задание №2

### Михаил Нефедов

####  МТР-153

На моем компьютере под Windows word2vec выдавал непонятную мне ошибку в 3.4 питоне, а в остальных версиях были проблемы со всем gensim'мом. Поэтому я выполнил задание на макбуке Анастасии Романовой. 

## Часть 1: word2vec


In [1]:
import pandas as pd
import re
import nltk.data
import warnings
import logging
import time
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec

In [2]:
train = pd.read_csv( "labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

print("Read %d labeled train reviews and %d unlabeled reviews\n" % (train["review"].size,  unlabeled_train["review"].size ))

Read 25000 labeled train reviews and 50000 unlabeled reviews



In [5]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review, 'lxml').get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [6]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [7]:
warnings.filterwarnings("ignore", category=UserWarning)
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set
Parsing sentences from unlabeled set


In [8]:
start = time.time() # Start time
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)

print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for training model: ", elapsed, "seconds.")

Training model...
Time taken for training model:  157.80408787727356 seconds.


In [7]:
model.accuracy("GoogleTestData.txt")

[{'correct': [('bangkok', 'thailand', 'berlin', 'germany'),
   ('bangkok', 'thailand', 'tokyo', 'japan'),
   ('beijing', 'china', 'paris', 'france'),
   ('beijing', 'china', 'tokyo', 'japan'),
   ('berlin', 'germany', 'paris', 'france'),
   ('cairo', 'egypt', 'paris', 'france'),
   ('london', 'england', 'paris', 'france'),
   ('moscow', 'russia', 'paris', 'france'),
   ('moscow', 'russia', 'tokyo', 'japan'),
   ('moscow', 'russia', 'berlin', 'germany'),
   ('moscow', 'russia', 'london', 'england'),
   ('paris', 'france', 'london', 'england'),
   ('rome', 'italy', 'tokyo', 'japan'),
   ('rome', 'italy', 'london', 'england'),
   ('rome', 'italy', 'paris', 'france'),
   ('tokyo', 'japan', 'london', 'england'),
   ('tokyo', 'japan', 'paris', 'france')],
  'incorrect': [('bangkok', 'thailand', 'beijing', 'china'),
   ('bangkok', 'thailand', 'cairo', 'egypt'),
   ('bangkok', 'thailand', 'havana', 'cuba'),
   ('bangkok', 'thailand', 'london', 'england'),
   ('bangkok', 'thailand', 'madrid', '

Большое количество неправильных ответов связано с тем, что а) размер корпуса недостаточно большой для представления всего языка б) корпус состоит из однотипных текстов 

Ниже только осмысленные и правильные ответы, но попадались и неправильные, напр. для pluto earth sun texas, выделялся pluto.

In [9]:
model.doesnt_match("dog cat cow rock".split())

'rock'

In [13]:
model.doesnt_match("dad mom bag son".split())

'bag'

In [15]:
model.doesnt_match("go sing come run ".split())

'sing'

In [24]:
model.doesnt_match("tomato onion mushroom go".split())

'go'

In [27]:
model.doesnt_match("iraq iran god egypt".split())

'god'

#### Я решил посмотреть близкие слова у оценочных прилагательных.

In [28]:
model.most_similar('wonderful')

[('great', 0.68970787525177),
 ('superb', 0.6670387983322144),
 ('brilliant', 0.6461779475212097),
 ('terrific', 0.6404646635055542),
 ('fantastic', 0.6282932758331299),
 ('excellent', 0.6127138137817383),
 ('amazing', 0.5903807878494263),
 ('marvelous', 0.5619683265686035),
 ('fine', 0.5490565299987793),
 ('outstanding', 0.5418260097503662)]

In [29]:
model.most_similar('terrific')

[('superb', 0.6818407773971558),
 ('fine', 0.6449187397956848),
 ('great', 0.6414417028427124),
 ('wonderful', 0.6404646635055542),
 ('excellent', 0.6355363130569458),
 ('outstanding', 0.6271907091140747),
 ('fantastic', 0.6124066114425659),
 ('marvelous', 0.5909610390663147),
 ('brilliant', 0.5762908458709717),
 ('solid', 0.5718175172805786)]

In [30]:
model.most_similar('boring')

[('dull', 0.7497252225875854),
 ('pointless', 0.672691822052002),
 ('tedious', 0.6587976217269897),
 ('predictable', 0.6138444542884827),
 ('uninteresting', 0.611487627029419),
 ('annoying', 0.5693108439445496),
 ('pretentious', 0.5618739128112793),
 ('stupid', 0.5614331960678101),
 ('repetitive', 0.5466095805168152),
 ('uneventful', 0.5441429615020752)]

In [33]:
model.most_similar('shitty')

[('cheezy', 0.543363630771637),
 ('tivo', 0.5416885614395142),
 ('ur', 0.5354318022727966),
 ('stunk', 0.5083209276199341),
 ('impressions', 0.5058048367500305),
 ('crud', 0.5048229098320007),
 ('vh', 0.5030849575996399),
 ('putrid', 0.502937376499176),
 ('nausea', 0.49917492270469666),
 ('waster', 0.4978787899017334)]

In [34]:
model.most_similar("crap")

[('garbage', 0.7649664878845215),
 ('trash', 0.7155179977416992),
 ('junk', 0.7065821290016174),
 ('rubbish', 0.6530443429946899),
 ('pile', 0.58770751953125),
 ('tripe', 0.5688697099685669),
 ('steaming', 0.5328880548477173),
 ('piece', 0.5273361206054688),
 ('utter', 0.5226682424545288),
 ('dreck', 0.510567307472229)]

## Часть 2

Я взял корпус текстов(точнее 2500 из 30к), который рассылался участникам соревнования по распознованию именованых сущностей. Там должны быть новостные тексты, но это не точно.

Так как в nltk нет русского лемматизатора, я вопользовался pymorphy2. Пунктуацию я убирал с помощью string.punctuation и там остались некоторые символы.

In [8]:
import nltk
import sys
import os
from math import log
import numpy as np
from time import time
from gensim import corpora, models, similarities
import logging


In [3]:
texts = []
for filename in os.listdir("/Users/anastasiaromanova/Desktop/nefmisha/result"):
    if filename.endswith("lem"):
        with open("/Users/anastasiaromanova/Desktop/nefmisha/result/" + filename,
                  encoding='utf-8') as f: 
            texts.append(f.read().split())

In [6]:
dictionary = corpora.Dictionary(texts)
print('Original: {}'.format(dictionary))
dictionary.filter_extremes(no_below = 5, no_above = 0.5, keep_n=None)
dictionary.save('news_txt.dict')
print('Filtered: {}'.format(dictionary))
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('news_text.mm', corpus)

Original: Dictionary(124401 unique tokens: ['«право', 'тнкbp', 'вагнеровский', 'эшмоловский', '«преизрядный»']...)
Filtered: Dictionary(24001 unique tokens: ['«право', 'тнкbp', 'богослов', 'reuters', 'поход']...)


In [7]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('news_text.mm', corpus)
print(corpus[0][0:20])

[(3, 1), (169, 1), (965, 1), (1128, 1), (2142, 1), (2372, 1), (2454, 1), (2544, 1), (2569, 1), (3330, 1), (3754, 1), (4279, 1), (4714, 1), (4889, 1), (5635, 1), (5860, 1), (6651, 1), (7309, 1), (7580, 2), (8046, 1)]


In [41]:
start = time()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

model = models.ldamodel.LdaModel(corpus, id2word = dictionary, num_topics=200, chunksize=50, update_every=3, passes=4)
print('Evaluation time: {}'.format((time()-start) / 60))

Evaluation time: 5.86073248386383


In [48]:
for topic in range(0, 50):
    for position in range(10):
        print(model.show_topic(topic)[position][1].center(20, ' '))
    print('--------------------')

       гоголь       
       газон        
       ильдар       
       кресло       
        илья        
     вежливость     
     оборонять      
    складываться    
      официант      
      земфира       
--------------------
       любовь       
       десять       
      двадцать      
     поколение      
      возраст       
        пара        
       тысяча       
     настоящий      
      столетие      
        или         
--------------------
      церковь       
        день        
       святой       
    православный    
      праздник      
        храм        
      верующий      
     священник      
      обитель       
    православие     
--------------------
       совет        
    организация     
    министерство    
       доклад       
    предложение     
     учреждение     
    соответствие    
       орган        
     подготовка     
      решение       
--------------------
        фонд        
       рубль        
       акция        
       индекс

Хорошие темы.


1. Выборы
-------------------- -
      процент       
       опрос        
       голос        
     избиратель     
       южный        
        два         
      опросить      
     результат      
     голосовать     
      половина      
      
 2. Матч
--------------------
        матч        
        счёт        
        игра        
       победа       
       после        
        два         
       минута       
      встреча       
       первое       
       серия       
 
 3. религия 
--------------------
      церковь       
        день        
       святой       
    православный    
      праздник      
        храм        
      верующий      
     священник      
      обитель       
    православие     
--------------------


Плохие
1. ???

--------------------
    супермаркет     
        ален        
       ровно        
      поистине      
      комплект      
      фотограф      
         аж         
       проезд       
        рейд        
      спокойно     
 2. ???
 
 --------------------
      анатолий      
       башня        
     лукашенко      
        раб         
       мотор        
       степан       
    оскорбление     
      фабрика       
       минск        
      концерн       
--------------------

In [44]:
some_document = corpus[0]
print(model[some_document])

[(9, 0.74223639237160255), (56, 0.012714207768120678), (74, 0.027974119735240985), (124, 0.016694785820426595), (152, 0.010691489361702224), (164, 0.087630415564135131), (198, 0.091792631931972152)]


In [39]:
def perplexity(model, corpus):
    corpus_length = 0
    log_likelihood = 0
    topic_profiles = model.state.get_lambda() / np.sum(model.state.get_lambda(), axis=1)[:, np.newaxis]
    for document in corpus:
        gamma, _ = model.inference([document])
        document_profile = gamma / np.sum(gamma)
        for term_id, term_count in document:
            corpus_length += term_count
            term_probability = np.dot(document_profile, topic_profiles[:, term_id])
            log_likelihood += term_count * log(term_probability)
    perplexity = np.exp(-log_likelihood / corpus_length)
    return perplexity

In [45]:
perplex = perplexity(model, corpus)
print("Perplexity: ", perplex)

Perplexity:  2329.14176317


Эта самая маленькая запутанность, которая у меня получилась из 3 попыток. В итоге получились не очень хорошие темы. Возможно, если убрать стоп-слова, очистить от всех знаков препинания, чисел и т.п., то темы будут более осмысленные. Также плохие результаты могут быть связаны с тем, что в корпусе могут быть тексты совершенно разных тематик и как-то их группировать бессмысленно. 